In [ ]:
import os
import glob
import numpy as np
import time
import json
import yaml
import pickle
import random
import pandas as pd
import argparse
import torch
import torch.nn as nn
from torchsummary import summary
from utils import Config
import torchvision

: 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [17]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    if (config["use_wandb"]):
        wandb.init(
            name=config["wandb_path"], project=config["wandb_project"], config=config, save_code=True)
    config = Config(**config)

### Feed Forward Neural Network

In [6]:
class FeedForward(torch.nn.Module):
    def __init__(self, config):
        super(FeedForward, self).__init__()

        self.config = config

        base_dim = (self.config.num_feat_cols + self.config.num_stat_cols) * self.config.context_len * self.config.x_dim * self.config.y_dim
        self.linear1 = nn.Linear(base_dim, base_dim*2)
        self.linear2 = nn.Linear(base_dim*2, base_dim)
        self.linear3 = nn.Linear(base_dim, self.config.num_feat_cols * 1 * self.config.x_dim * self.config.y_dim)

        self.internal_activation = nn.ReLU()

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=4)
        x = self.internal_activation(self.linear1(x))
        x = self.internal_activation(self.linear2(x))
        x = self.linear3(x)
        x = torch.reshape(x, (-1, 1, self.config.num_feat_cols, self.config.x_dim, self.config.y_dim))

        return x

In [32]:
summary(model, input_size=(5, 5, 10, 10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 5000]      12,505,000
              ReLU-2                 [-1, 5000]               0
            Linear-3                 [-1, 2500]      12,502,500
              ReLU-4                 [-1, 2500]               0
            Linear-5                  [-1, 300]         750,300
Total params: 25,757,800
Trainable params: 25,757,800
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.12
Params size (MB): 98.26
Estimated Total Size (MB): 98.38
----------------------------------------------------------------


In [30]:
model = FeedForward(config).to(device)
print(model)

FeedForward(
  (linear1): Linear(in_features=2500, out_features=5000, bias=True)
  (linear2): Linear(in_features=5000, out_features=2500, bias=True)
  (linear3): Linear(in_features=2500, out_features=300, bias=True)
  (internal_activation): ReLU()
)


### Dilated CNN

In [12]:
class DilatedCNN(torch.nn.Module):
    def __init__(self):
        super(DilatedCNN, self).__init__()
        
        self.fclayers = nn.Sequential(
        nn.Conv2d(in_channels=5, out_channels=256, kernel_size = 5, stride = 1, padding= 'valid', dilation = 1),
        nn.Conv2d(in_channels=256, out_channels=128, kernel_size = 3, stride = 1, padding= 'valid', dilation = 2),
        nn.Conv2d(in_channels=128, out_channels=64, kernel_size = 1, stride = 1, padding= 'valid', dilation = 4),
        nn.Flatten(start_dim=0),
        nn.Linear(1280, 1000),
        nn.ReLU(),
        nn.Linear(1000, 300),
        nn.ReLU(),
        )
    def forward(self, x):
        print("the shape is", x.shape)
        x = self.fclayers(x)
        x = torch.reshape(x, (1, 3, 10, 10))
        return x

In [13]:
model = DilatedCNN().to(device)
print(model)

DilatedCNN(
  (fclayers): Sequential(
    (0): Conv2d(5, 256, kernel_size=(5, 5), stride=(1, 1), padding=valid)
    (1): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=valid, dilation=(2, 2))
    (2): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=valid, dilation=(4, 4))
    (3): Flatten(start_dim=0, end_dim=-1)
    (4): Linear(in_features=1280, out_features=1000, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1000, out_features=300, bias=True)
    (7): ReLU()
  )
)


In [14]:
x = torch.randn(5, 5, 10, 10)
y = model(x)
print(y.shape)

the shape is torch.Size([5, 5, 10, 10])
torch.Size([1, 3, 10, 10])


In [97]:
x = torch.randn(5, 5, 10, 10)
print(x.shape)
x = nn.Conv2d(in_channels=5, out_channels=256, kernel_size = 5, stride = 1, padding= 'valid', dilation = 1)(x)
print(x.shape)
x = nn.Conv2d(in_channels=256, out_channels=128, kernel_size = 3, stride = 1, padding= 'valid', dilation = 2)(x)
print(x.shape)
x = nn.Conv2d(in_channels=128, out_channels=64, kernel_size = 1, stride = 1, padding= 'valid', dilation = 4)(x)
y = nn.Flatten(start_dim=0)(x)
print(x.shape)
print("y = ", y.shape)
x = nn.Linear(1280,1000)(y)
print(x.shape)
x = nn.Linear(1000, 300)(x)
print(x.shape)
x = torch.reshape(x, (1, 3, 10, 10))
print(x.shape)

torch.Size([5, 5, 10, 10])
torch.Size([5, 256, 6, 6])
torch.Size([5, 128, 2, 2])
torch.Size([5, 64, 2, 2])
y =  torch.Size([1280])
torch.Size([1000])
torch.Size([300])
torch.Size([1, 3, 10, 10])


### ConvLSTM

In [15]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        #self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.padding = 'valid'
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

In [18]:
class ConvLSTM(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        kernel_size = [(5,5), (3,3), (1,1)]

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                print(cur_layer_input.shape)
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [23]:
model = ConvLSTM(input_dim=5,
                 hidden_dim=[64, 64, 64],
                 kernel_size=(3, 3),
                 num_layers=3,
                 batch_first=False,
                 bias=True,
                 return_all_layers=False)

summary(model, (5, 5, 10, 10))

torch.Size([5, 2, 5, 10, 10])


RuntimeError: The size of tensor a (6) must match the size of tensor b (10) at non-singleton dimension 3

In [ ]:
x = torch.rand(1, 5, 5, 10, 10)
y = model(x)
print(y.shape)

### U-Net

In [28]:
import torch
import torch.nn as nn

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding='same'),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding='same'),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self):
        super().__init__()
                
        self.dconv_down1 = double_conv(5, 12)
        self.dconv_down2 = double_conv(12, 24)
        self.dconv_down3 = double_conv(24, 24)
        
        self.maxpool = nn.MaxPool2d(2)

        self.upsample1 = nn.Upsample(scale_factor=2.5, mode='bilinear', align_corners=True)        
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.dconv_up3 = double_conv(48, 24)
        self.dconv_up2 = double_conv(24, 12)
        
        self.conv_last = nn.Conv2d(12, 3, 1)
        
        
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)        
        conv3 = self.dconv_down3(x)
        x = self.upsample1(x)  
        #print(x.shape, conv2.shape)      
        x = torch.cat([x, conv2], dim=1)
        #print(x.shape)
        x = self.dconv_up3(x)
        #print("before upsample", x.shape)
        x = self.upsample2(x)
        x = nn.Conv2d(24, 12, 3, padding='same')(x)      
        #print(x.shape, conv1.shape)
        x = torch.cat([x, conv1], dim=1)
        #print(x.shape)
        x = self.dconv_up2(x)
        out = self.conv_last(x)
        #print(out.shape)
        return out

In [29]:
unet = UNet()
summary(UNet(), (5, 10, 10))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 10, 10]             552
              ReLU-2           [-1, 12, 10, 10]               0
            Conv2d-3           [-1, 12, 10, 10]           1,308
              ReLU-4           [-1, 12, 10, 10]               0
         MaxPool2d-5             [-1, 12, 5, 5]               0
            Conv2d-6             [-1, 24, 5, 5]           2,616
              ReLU-7             [-1, 24, 5, 5]               0
            Conv2d-8             [-1, 24, 5, 5]           5,208
              ReLU-9             [-1, 24, 5, 5]               0
        MaxPool2d-10             [-1, 24, 2, 2]               0
           Conv2d-11             [-1, 24, 2, 2]           5,208
             ReLU-12             [-1, 24, 2, 2]               0
           Conv2d-13             [-1, 24, 2, 2]           5,208
             ReLU-14             [-1, 2

### Deep AR

In [ ]:
!pip install pytorch_forecasting

In [ ]:
training = TimeSeriesDataSet(
    # data[lambda x: x.time_idx <= training_cutoff],
    # time_idx="time_idx",
    # target="value",
    categorical_encoders={"series": NaNLabelEncoder().fit(data.series)},
    group_ids=["series"],
    static_categoricals=[
        "series"
    ],  # as we plan to forecast correlations, it is important to use series characteristics (e.g. a series identifier)
    time_varying_unknown_reals=["value"],
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,
)

In [ ]:
from pytorch_forecasting import DeepAR
model = DeepAR.from_dataset(
    training,
    learning_rate=0.1,
    log_interval=10,
    log_val_interval=1,
    hidden_size=30,
    rnn_layers=2,
    loss=MultivariateNormalDistributionLoss(rank=30),
)

In [6]:
x = torch.randn(5, 5, 10, 10)
y = model(x)
print(y.shape)
# summary(DeepAR(), (5, 10, 10))

c:\Users\Aviral\.conda\envs\mtp\lib\site-packages\pytorch_lightning\utilities\parsing.py:262: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
c:\Users\Aviral\.conda\envs\mtp\lib\site-packages\pytorch_lightning\utilities\parsing.py:262: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


AssertionError: target None has to be real